In [1]:
#modules
import matplotlib.pyplot as plt
import xgboost as xgb
from matplotlib import rcParams
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
!pip install scikit_optimize
from scipy.stats import spearmanr
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
seed = 42

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 4.8 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
train_data=pd.read_csv("/content/drive/MyDrive/novozymes-enzyme-stability-prediction/train.csv")

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31104 non-null  float64
 3   data_source       28043 non-null  object 
 4   tm                31390 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


#data preprocessing

In [5]:
#feature-1 Length of the protien 
!pip3 install protlearn
import protlearn as pl 
from protlearn.features import length, entropy, aaindex1, atc, aac

# Why Protlearn Module ?
"""
A technique for extracting features from protein sequences is called protolearn. 
It is a free Python tool that enables users to effectively extract amino acid sequence features from proteins or peptides. 
These features may then be utilized for a range of downstream machine learning tasks. 
Currently, protlearn consists of three primary modules: 

dimension reduction and feature extraction during preprocessing 
The Preprocessing area enables the user to prepare the datasets in accordance with certain requirements, such as removing sequences that include unnatural amino acids or changing alphabetical strings into integers. 
The dataset's amino acid sequence features, such as amino acid composition or AAIndex-based physicochemical attributes, can then be computed using the Feature Extraction section. 
Protlearn presently offers 21 such functions in total.
The dimensionality of the computed features can be reduced using feature selection techniques, which lessens redundant computation and lessens the computational load on the classifiers.
 

From my research from the protien sequence the important elements we can extract can be done by using the below functions:

Functions Used:

1. Amino acid composition: (aac)
This function returns the frequency of amino acids for each sequence in the dataset. aac is calculated as follows:
𝑎𝑎𝑐(𝑖) = 𝑛𝑖/N
    
protlearn.features.aac(X, *, method='relative', remove_zero_cols=False, start=1, ˓→end=None)
 where i denotes the 20 amino acid residues, ni is the frequency of each residue i, and N is the total number of residues in the sequence.


2. AAIndex1-based physicochemical properties.
AAindex1 ver.9.2 (release Feb, 2017) is a set of 20 numerical values representing various physicochemical and bio- logical properties of amino acids. Currently, it contains 566 indices, of which 553 contain no NaNs. The indices will be collected for each amino acid in the sequence, then averaged across the sequence.
aaindex1 is calculated as follows:

∑︀𝑁𝑛=1 (𝐴𝐴𝑖𝑛𝑑𝑒𝑥𝑖(𝑎𝑎𝑛) *𝑎𝑎𝑖𝑛𝑑𝑒𝑥1(𝑖))/ 𝑁

where i denotes the 566 AAIndex1 indices, aan denotes the amino acid at position n, and N is the total number of residues in the sequence.

3. protlearn.features.atc(X, *, method='relative', start=1, end=None)
Atomic and bond composition.

This function returns the sum of atomic and bond compositions for each amino acid sequence. The atomic features are comprised of five atoms (C, H, N, O, and S), and the bond features are comprised of total bonds, single bonds, and double bonds.
atc is calculated as follows:
𝑎𝑡𝑜𝑚𝑠(𝑖) = 𝑛𝑖/𝑁
"""


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 7.6 MB/s 
     |████████████████████████████████| 3.1 MB 50.1 MB/s 


"\nA technique for extracting features from protein sequences is called protolearn. \nIt is a free Python tool that enables users to effectively extract amino acid sequence features from proteins or peptides. \nThese features may then be utilized for a range of downstream machine learning tasks. \nCurrently, protlearn consists of three primary modules: \n\ndimension reduction and feature extraction during preprocessing \nThe Preprocessing area enables the user to prepare the datasets in accordance with certain requirements, such as removing sequences that include unnatural amino acids or changing alphabetical strings into integers. \nThe dataset's amino acid sequence features, such as amino acid composition or AAIndex-based physicochemical attributes, can then be computed using the Feature Extraction section. \nProtlearn presently offers 21 such functions in total.\nThe dimensionality of the computed features can be reduced using feature selection techniques, which lessens redundant co

In [6]:
def data_preprocessing(train_data):
  import pandas as pd 
  def length_of_protien(df):
    protien=[]
    for i in df["protein_sequence"]:
      protien.append(len(i))
    return protien 
    #feature - 2 Entropy 

  def entropy_finder(df):
    entropy=[]
    for i in df["protein_sequence"]:
      entropy.append(i)
    return entropy 
  def length_of_protien(df):
    protien=[]
    for i in df["protein_sequence"]:
      protien.append(len(i))
    return protien 
  #feature - 3
  atoms, bonds = atc(train_data['protein_sequence'].tolist())
  comp, aa = aac(train_data['protein_sequence'].tolist())
  # Composition in the protien sequences 

  # Atoms in the protien sequences 

  # Bonds in the protien sequencs 

  def Form_a_data(keys,values):
    sample_dict = dict()
    dict_keys = keys
    for i in range(len(dict_keys)):
        sample_dict[dict_keys[i]] = [v[i] for v in values]
    return pd.DataFrame(sample_dict)
  compostion_data=Form_a_data([*aa],comp)
  atom_Data=Form_a_data(['Carbon','Hydrogen','Nitrogen','Oxygen','Sulphur'],atoms)
  bonds_data=Form_a_data(['total_bonds','single','double'],bonds)
  train_data["protien_length"]=length_of_protien(train_data)

  train_data["entropy"]=entropy_finder(train_data)

  train_data["protien_length"]=length_of_protien(train_data)

  main_data=pd.concat([train_data,compostion_data,atom_Data,bonds_data],axis=1)
  main_data.drop(["protein_sequence","data_source","seq_id"],axis=1,inplace=True)
  main_data.drop(["entropy"],axis=1,inplace=True)
  return main_data
main_data=data_preprocessing(train_data)

In [7]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pH              31104 non-null  float64
 1   tm              31390 non-null  float64
 2   protien_length  31390 non-null  int64  
 3   A               31390 non-null  float64
 4   C               31390 non-null  float64
 5   D               31390 non-null  float64
 6   E               31390 non-null  float64
 7   F               31390 non-null  float64
 8   G               31390 non-null  float64
 9   H               31390 non-null  float64
 10  I               31390 non-null  float64
 11  K               31390 non-null  float64
 12  L               31390 non-null  float64
 13  M               31390 non-null  float64
 14  N               31390 non-null  float64
 15  P               31390 non-null  float64
 16  Q               31390 non-null  float64
 17  R               31390 non-null 